In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Post-Traumatic_Stress_Disorder/GSE81761'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene Expression Pathways Implicated in Posttraumatic Stress Disorder and Symptomatic Improvement"
!Series_summary	"Military service members often return from deployment with a multiplicity of injuries, including mild traumatic brain injury, depression, and sleep disorders, which obsures diagnosis of PTSD symptoms and complicates treatment of PTSD. In order to understand the biological mechanisms underlying PTSD, gene expression profiles of military service members with and without PTSD were compared. Additionally, gene expression was examined based on intrusion symptoms, a distinct subtype of PTSD symptoms, and on improvement of PTSD symptoms at a three month follow up. RNA was extracted from blood samples and hybridized to the HG-U133_Plus_2 Affymetrix chip."
!Series_overall_design	"Gene expression of subjects with PTSD (n=39) were compared to controls without PTSD (n=27) at baseline. Further analysis of gene expression for subjects with PTSD at 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Based on the provided background information this dataset likely contains gene expression data

# Determine availability of variables
trait_row = 1 if len(set(['case/control: PTSD', 'case/control: No PTSD'])) > 1 else None
age_row = 5 if len(set(['age: 30', 'age: 38', 'age: 39', 'age: 23', 'age: 48', 'age: 49', 'age: 34', 'age: 33', 'age: 45', 'age: 25',
                        'age: 22', 'age: 46', 'age: 35', 'age: 36', 'age: 43', 'age: 26', 'age: 27', 'age: 28', 'age: 29',
                        'age: 41', 'age: 44', 'age: 31', 'age: 42', 'age: 21', 'age: 37', 'age: 52', 'age: 24', 'age: 32'])) > 1 else None
gender_row = 4 if len(set(['Sex: Male', 'Sex: Female'])) > 1 else None

# Define conversion functions
def convert_trait(value):
    value = value.split(':')[-1].strip()
    if value == "PTSD":
        return 1
    elif value == "No PTSD":
        return 0
    return None

def convert_age(value):
    try:
        return int(value.split(':')[-1].strip())
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(':')[-1].strip()
    if value == "Male":
        return 1
    elif value == "Female":
        return 0
    return None

# Save cohort information
save_cohort_info('GSE81761', './preprocessed/Post-Traumatic_Stress_Disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait_row is not None
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Post-Traumatic_Stress_Disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Post-Traumatic_Stress_Disorder/trait_data/GSE81761.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2175165': [1, 30, 1], 'GSM2175166': [0, 38, 1], 'GSM2175167': [1, 39, 1], 'GSM2175168': [1, 38, 1], 'GSM2175169': [1, 23, 1], 'GSM2175170': [0, 48, 1], 'GSM2175171': [0, 49, 1], 'GSM2175172': [0, 34, 1], 'GSM2175173': [1, 33, 1], 'GSM2175174': [1, 45, 1], 'GSM2175175': [1, 25, 1], 'GSM2175176': [0, 25, 1], 'GSM2175177': [1, 30, 1], 'GSM2175178': [1, 39, 1], 'GSM2175179': [1, 23, 1], 'GSM2175180': [1, 22, 1], 'GSM2175181': [1, 46, 1], 'GSM2175182': [0, 35, 1], 'GSM2175183': [0, 22, 1], 'GSM2175184': [1, 23, 1], 'GSM2175185': [0, 48, 1], 'GSM2175186': [1, 23, 1], 'GSM2175187': [0, 49, 1], 'GSM2175188': [1, 38, 1], 'GSM2175189': [1, 25, 1], 'GSM2175190': [1, 33, 1], 'GSM2175191': [1, 30, 1], 'GSM2175192': [0, 36, 1], 'GSM2175193': [0, 43, 1], 'GSM2175194': [0, 34, 1], 'GSM2175195': [1, 22, 1], 'GSM2175196': [0, 26, 1], 'GSM2175197': [1, 46, 1], 'GSM2175198': [1, 39, 0], 'GSM2175199': [1, 27, 1], 'GSM2175200': [1, 23, 1], 'GSM2175201': [0, 28, 1], 'GSM2175202': [0, 22, 1], 'GSM2175203

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])
    

# The given identifiers are Affymetrix probe set IDs, not standard human gene symbols.


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify relevant keys for mapping
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe for mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to the gene expression data and name it "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Post-Traumatic_Stress_Disorder/gene_data/GSE81761.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Post-Traumatic_Stress_Disorder')

# 4. Save the cohort information.
save_cohort_info('GSE81761', './preprocessed/Post-Traumatic_Stress_Disorder/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Post-Traumatic_Stress_Disorder/GSE81761.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Post-Traumatic_Stress_Disorder', the least common label is '0.0' with 47 occurrences. This represents 43.12% of the dataset.
The distribution of the feature 'Post-Traumatic_Stress_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 26.0
  50% (Median): 33.0
  75%: 39.0
Min: 21.0
Max: 52.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 5 occurrences. This represents 4.59% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

